In [1]:
from rateLimiter import RateLimiter, set_log_level
import cloudscraper
from tqdm import tqdm
import logging
import timeit


In [2]:
set_log_level(logging.DEBUG)

2025-03-14 11:05:56,891 - rateLimiter - INFO - Log level set to: 10


In [3]:
types = [
    "Advanced Aged Cuffs", 
    "Advanced Altar Robe", 
    "Advanced Anchorite Garb", 
    "Advanced Antler Focus", 
    "Advanced Barrier Quarterstaff", 
    "Advanced Beaded Circlet", 
    "Advanced Blazon Crest Shield",
    "Advanced Bolstered Mitts", 
    "Advanced Bombard Crossbow", 
    "Advanced Bound Bracers", 
    "Advanced Braced Sabatons", 
    "Advanced Braced Tower Shield",
    "Advanced Brigand Mace", 
    "Advanced Brimmed Helm", 
    "Advanced Bronze Greaves", 
    "Advanced Chain Mail", 
    "Advanced Chain Tiara", 
    "Advanced Chiseled Targe", 
    "Advanced Cloaked Mail", 
    "Advanced Composite Bow", 
    "Advanced Construct Hammer", 
    "Advanced Cowled Helm", 
    "Advanced Crackling Quarterstaff", 
    "Advanced Crescent Quarterstaff", 
    "Advanced Crescent Targe", 
    "Advanced Crumbling Maul", 
    "Advanced Crystal Focus", 
    "Advanced Cultist Bow", 
    "Advanced Cultist Crown", 
    "Advanced Cultist Greathammer", 
    "Advanced Doubled Gauntlets", 
    "Advanced Dualstring Bow", 
    "Advanced Dyad Crossbow", 
    "Advanced Edged Buckler", 
    "Advanced Effigial Tower Shield",
    "Advanced Elite Greathelm", 
    "Advanced Emblem Crest Shield",
    "Advanced Embossed Boots", 
    "Advanced Explorer Armour", 
    "Advanced Face Mask", 
    "Advanced Feathered Robe", 
    "Advanced Feathered Sandals", 
    "Advanced Feathered Targe", 
    "Advanced Feathered Tiara", 
    "Advanced Felt Cap", 
    "Advanced Firm Bracers", 
    "Advanced Forge Maul", 
    "Advanced Forlorn Crossbow", 
    "Advanced Frayed Shoes", 
    "Advanced Full Plate", 
    "Advanced Fur Plate", 
    "Advanced Gauze Wraps", 
    "Advanced Goldcast Cuffs", 
    "Advanced Gothic Quarterstaff", 
    "Advanced Guarded Helm", 
    "Advanced Hardwood Targe", 
    "Advanced Heavy Crown", 
    "Advanced Heraldric Tower Shield",
    "Advanced Hermit Garb"
]

In [4]:
url = "http://www.pathofexile.com/api/trade2/search/Standard"

payload = {
    "query": {
        "status": { "option": "online" },
        "type": "Heavy Belt",
        "stats": [
            {
                "type": "and",
                "filters": []
            }
        ]
    },
    "sort": { "price": "asc" }
}
headers = {"content-type": "application/json"}
session = RateLimiter(debug=True)

In [5]:
session.post(
    url,
    payload=payload,
    headers=headers
)

2025-03-14 11:05:59,887 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): www.pathofexile.com:80
2025-03-14 11:06:00,146 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None
2025-03-14 11:06:00,147 - rateLimiter - DEBUG - Waiting for 14.0 | RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=1:window=300:penalty=0>


< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.9149.622 Mobile Safari/537.36
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
< Accept-Language: en-US,en;q=0.9
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 134
< 
< {"query": {"status": {"option": "online"}, "type": "Heavy Belt", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Fri, 14 Mar 2025 16:05:53 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,1:60:0,

2025-03-14 11:06:14,148 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,1:60:0,1:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=1:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=1:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 14, 11, 6, 0, 147133), 'prev_state': None, 'prev_rate_limits': None}})


<Response [200]>

In [6]:
session.session.get_tokens(url)

2025-03-14 11:06:32,663 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): www.pathofexile.com:80
2025-03-14 11:06:32,803 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "GET /api/trade2/search/Standard HTTP/11" 301 None
2025-03-14 11:06:32,804 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.pathofexile.com:443
2025-03-14 11:06:32,919 - urllib3.connectionpool - DEBUG - https://www.pathofexile.com:443 "GET /api/trade2/search/Standard HTTP/11" 401 None
2025-03-14 11:06:32,920 - root - ERROR - "http://www.pathofexile.com/api/trade2/search/Standard" returned an error. Could not collect tokens.


HTTPError: 401 Client Error: Unauthorized for url: https://www.pathofexile.com/api/trade2/search/Standard

In [6]:
for i in tqdm(types):
    payload["query"]["type"] = i
    res = session.post(
        url,
        payload=payload,
        headers=headers
    )

  0%|          | 0/59 [00:00<?, ?it/s]2025-03-14 10:33:12,146 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None
2025-03-14 10:33:12,147 - rateLimiter - DEBUG - Waiting for 12.0 | RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>


< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:53.0) Gecko/20100101 Firefox/53.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 143
< 
< {"query": {"status": {"option": "online"}, "type": "Advanced Aged Cuffs", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Fri, 14 Mar 2025 15:33:05 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,2:60:0,2:300:0
> X-Frame-Options: SAMEORIGIN
> Strict-Transport-Security:

2025-03-14 10:33:24,148 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,2:60:0,2:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 14, 10, 33, 12, 147525), 'prev_state': '1:10:0,1:60:0,1:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=1:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=1:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
  2%|▏         | 1/59 [00:12<11:44, 12.14s/it]2025-03-14 10:33:24,301 - urllib3.connectionp

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:53.0) Gecko/20100101 Firefox/53.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 143
< 
< {"query": {"status": {"option": "online"}, "type": "Advanced Altar Robe", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Fri, 14 Mar 2025 15:33:17 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,3:60:0,3:300:0
> X-Frame-Options: SAMEORIGIN
> Strict-Transport-Security:

2025-03-14 10:33:36,304 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,3:60:0,3:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=3:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=3:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 14, 10, 33, 24, 302638), 'prev_state': '1:10:0,2:60:0,2:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
  3%|▎         | 2/59 [00:24<11:32, 12.15s/it]2025-03-14 10:33:36,536 - urllib3.connectionp

...

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:53.0) Gecko/20100101 Firefox/53.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 144
< 
< {"query": {"status": {"option": "online"}, "type": "Advanced Cultist Bow", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Fri, 14 Mar 2025 15:38:33 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,4:60:0,23:300:0
> X-Frame-Options: SAMEORIGIN
> Strict-Transport-Securit

2025-03-14 10:38:52,688 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,4:60:0,23:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=23:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=4:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 14, 10, 38, 40, 686204), 'prev_state': '1:10:0,5:60:0,22:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=22:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=5:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 47%|████▋     | 28/59 [05:40<06:17, 12.18s/it]2025-03-14 10:38:52,810 - urllib3.connec

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:53.0) Gecko/20100101 Firefox/53.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 146
< 
< {"query": {"status": {"option": "online"}, "type": "Advanced Cultist Crown", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Fri, 14 Mar 2025 15:38:45 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,4:60:0,24:300:0
> X-Frame-Options: SAMEORIGIN
> Strict-Transport-Secur

2025-03-14 10:39:04,812 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,4:60:0,24:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=24:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=4:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 14, 10, 38, 52, 811785), 'prev_state': '1:10:0,4:60:0,23:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=23:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=4:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 49%|████▉     | 29/59 [05:52<06:04, 12.16s/it]2025-03-14 10:39:04,845 - urllib3.connec

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:53.0) Gecko/20100101 Firefox/53.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 152
< 
< {"query": {"status": {"option": "online"}, "type": "Advanced Cultist Greathammer", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 429 Too Many Requests
> Date: Fri, 14 Mar 2025 15:38:57 GMT
> Content-Type: application/json
> Content-Length: 84
> Connection: keep-alive
> Retry-After: 600
> Vary: Accept-Encoding
> Server: cloudflare
> CF-RAY: 9204dc6fd932d322-MCI
> 
{
    "error": {
        "code": 3,
        "message": "Rate limit exceeded"
    }
}


In [23]:
res = session.session.post(
        url,
        json=payload,
        headers=headers
    )
print(res.status_code)
print(res.text)
print(res.headers)


2025-03-13 21:06:23,872 - urllib3.connectionpool - DEBUG - Resetting dropped connection: www.pathofexile.com
2025-03-13 21:06:24,063 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None


200
{"id":"aDaM4V3ue","complexity":5,"result":["fa0665087835d53ea92d73e5684fb19056b31f6a9e6dce6c74b44de289946bd5","a81b2e3ff26541be20bf2affa42077bc82c7c75426efebcf4a84d2b7bf1dd43b","943313b1a5345a82e4bbe0e4dbe3ca90e1d140c87a682d632ef5b2d74a031114","76845362ff73808298f7a1aa53a318d1fcbb8de93d386d5733b4876399e8ca47","fe013e1f4edabe371fc2c71162dfb6c850fcdcad27f334d1cd5a4fdc17d1500a","e610aa90e9b075ff59c14dc3a89a569375d0dc701254e66e42f85b089526e105","0b642c7894480411cd0f976f7898c0706789c01ebbf12dc18b3b600a9e523f9b","4dd58b9011548c04979c6e4ae7a13c5d453d7159acc82a0aded315d5f3f181f6","cc67b5a8d36b47369527666c89a140f8c9b19eaeb535480bcf55fb3a9633d5e0","96915e45f2f015366879bed35ee656ff83336afcfe8d591cb6cf00d38814c495","7476153e864807b4ad4402c61b6719e5d955062ff2393c7623f77bc5e26a8f07","d708a0520ad799ed057578483220b72f705d09dd5a083b626d47f2a903f0ec32","1f90566df8f2cc8314b182286b4566da959e496441ffcf707104aeaf52887f98","17ada987bcca7748b1f364e96dab3b5f9d3a8403032bc9b77379870ecd7453a5","7be5473d401b4c